# Use Convolutional Neural Network for Biome Classification
-- by jenny
from
[https://github.com/aymericdamien/TensorFlow-Examples/blob/master/examples/3_NeuralNetworks/convolutional_network.py](https://github.com/aymericdamien/TensorFlow-Examples/blob/master/examples/3_NeuralNetworks/convolutional_network.py)
[A Guide to TF Layers: Building a Convolutional Neural Network Contents](https://www.tensorflow.org/tutorials/layers)

In [1]:
%pylab inline
import tensorflow as tf


Populating the interactive namespace from numpy and matplotlib


In [2]:

IMAGE_HEIGHT = 200
IMAGE_WIDTH = 320
CHANNEL = 3

# Parameters
learning_rate = 0.01
training_iters = 200
batch_size = 1
display_step = 10

# Network Parameters
n_input = IMAGE_HEIGHT*IMAGE_WIDTH*CHANNEL # MNIST data input (img shape: 28*28)
n_classes = 2 # MNIST total classes (0-9 digits)
dropout = 0.75 # Dropout, probability to keep units


In [3]:
classes ={0:"desert",1:"forest",2:"jungle", 3: "mesa",4:"eh"}
tfrecords_train_filename = '/notebooks/Minecraft-AI/mc-data/mesa_3_vs_forest_1_train.tfrecords'
tfrecords_test_filename = '/notebooks/Minecraft-AI/mc-data/mesa_3_vs_forest_1_test.tfrecords'

## load data
### healper function for reading and decoding data in tfrecord file

In [4]:
def scaleImg(img):
    scaled_img = img.astype(numpy.float32)
    return scaled_img /255

In [27]:
def read_and_decode(filename_queue):
    
    reader = tf.TFRecordReader()

    _, serialized_example = reader.read(filename_queue)

    features = tf.parse_single_example(
      serialized_example,
      # Defaults are not specified since both keys are required.
      features={
        'height': tf.FixedLenFeature([], tf.int64),
        'width': tf.FixedLenFeature([], tf.int64),
        'image_raw': tf.FixedLenFeature([], tf.string),
        'mask_raw': tf.FixedLenFeature([], tf.string),
        'label': tf.FixedLenFeature([], tf.int64)
        })

    # Convert from a scalar string tensor (whose single string has
    # length mnist.IMAGE_PIXELS) to a uint8 tensor with shape
    # [mnist.IMAGE_PIXELS].
    image = tf.decode_raw(features['image_raw'], tf.uint8)
    print("image shape")
    print(image.shape)
    annotation = tf.decode_raw(features['mask_raw'], tf.uint8)
    label =  tf.cast(features['label'], tf.int32)
    
    height = tf.cast(features['height'], tf.int32)
    width = tf.cast(features['width'], tf.int32)
    
#     tf.stack equal to tf.stack([x, y, z]) = np.asarray([x, y, z])
    image_shape = tf.stack([height, width, 3])
    annotation_shape = tf.stack([height, width, 1])
    # not reshapeing anymore
    image = tf.reshape(image, image_shape)
    annotation = tf.reshape(annotation, annotation_shape)
    
    image_size_const = tf.constant((IMAGE_HEIGHT, IMAGE_WIDTH, 3), dtype=tf.int32)
    annotation_size_const = tf.constant((IMAGE_HEIGHT, IMAGE_WIDTH, 1), dtype=tf.int32)
    
    # Random transformations can be put here: right before you crop images
    # to predefined size. To get more information look at the stackoverflow
    # question linked above.
    
    resized_image = tf.image.resize_image_with_crop_or_pad(image=image,
                                           target_height=IMAGE_HEIGHT,
                                           target_width=IMAGE_WIDTH)
    
    resized_annotation = tf.image.resize_image_with_crop_or_pad(image=annotation,
                                           target_height=IMAGE_HEIGHT,
                                           target_width=IMAGE_WIDTH)
    
    
    images, annotations,label = tf.train.shuffle_batch( [resized_image, resized_annotation,label],
                                                 batch_size=batch_size,
                                                 capacity=5000,
                                                 num_threads=3,
                                                 min_after_dequeue=10)
    
    return images, annotations, label

## Config CNN

If your input is of size w1\*h1\*d1 and the size of the filter is f\*f with stride S. Then the output sizes w2\*h2\*d2 will be:
```
w2= (w1-f)/S +1

h2=(h1-f)/S +1

d2=d1
```

In [28]:
# tf Graph input
x = tf.placeholder(tf.float32, [None, n_input])
print(x.shape)
y = tf.placeholder(tf.float32, [None, n_classes])
keep_prob = tf.placeholder(tf.float32) #dropout (keep probability)


(?, 192000)


In [29]:
# Create some wrappers for simplicity
def conv2d(x, W, b, strides=2):
    # Conv2D wrapper, with bias and relu activation
    print("x shape before", x.shape)
    print("y shape ",y.shape)
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='VALID')
    print("x shape after conv2d ",x.shape)
    x = tf.nn.bias_add(x, b)
    print("x shape after bias_add ",x.shape)
    return tf.nn.relu(x)


def maxpool2d(x, k=2):
    # MaxPool2D wrapper
    return tf.nn.max_pool(x, ksize=[1, k, k, 1], strides=[1, k, k, 1],
                          padding='SAME')

# Create model
def conv_net(x, weights, biases, dropout):
    # Reshape input picture
    x = tf.reshape(x, shape=[-1, IMAGE_HEIGHT, IMAGE_WIDTH, 3])
    print("x initial reshape ", x.shape)
    # Convolution Layer
    conv1 = conv2d(x, weights['wc1'], biases['bc1'])
    # Max Pooling (down-sampling)
    conv1 = maxpool2d(conv1, k=2)

    # Convolution Layer
    conv2 = conv2d(conv1, weights['wc2'], biases['bc2'])
    # Max Pooling (down-sampling)
    conv2 = maxpool2d(conv2, k=2)

    # Fully connected layer
    # Reshape conv2 output to fit fully connected layer input
    fc1 = tf.reshape(conv2, [-1, weights['wd1'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    fc1 = tf.nn.relu(fc1)
    # Apply Dropout
    fc1 = tf.nn.dropout(fc1, dropout)

    # Output, class prediction
    out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
    return out



In [30]:
# Store layers weight & bias
weights = {
    # 5x5 conv, 1 input, 32 outputs
    'wc1': tf.Variable(tf.random_normal([5, 5, 3, 32])),
    # 5x5 conv, 32 inputs, 64 outputs
    'wc2': tf.Variable(tf.random_normal([5, 5, 32, 64])),
    # fully connected, 7*7*64 inputs, 1024 outputs
    'wd1': tf.Variable(tf.random_normal([7*7*64, 1024])),
    # 1024 inputs, 10 outputs (class prediction)
    'out': tf.Variable(tf.random_normal([1024, n_classes]))
}

biases = {
    'bc1': tf.Variable(tf.random_normal([32])),
    'bc2': tf.Variable(tf.random_normal([64])),
    'bd1': tf.Variable(tf.random_normal([1024])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}


In [31]:

# Construct model
pred = conv_net(x, weights, biases, keep_prob)

# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)


('x initial reshape ', TensorShape([Dimension(None), Dimension(200), Dimension(320), Dimension(3)]))
('x shape before', TensorShape([Dimension(None), Dimension(200), Dimension(320), Dimension(3)]))
('y shape ', TensorShape([Dimension(None), Dimension(2)]))
('x shape after conv2d ', TensorShape([Dimension(None), Dimension(98), Dimension(158), Dimension(32)]))
('x shape after bias_add ', TensorShape([Dimension(None), Dimension(98), Dimension(158), Dimension(32)]))
('x shape before', TensorShape([Dimension(None), Dimension(49), Dimension(79), Dimension(32)]))
('y shape ', TensorShape([Dimension(None), Dimension(2)]))
('x shape after conv2d ', TensorShape([Dimension(None), Dimension(23), Dimension(38), Dimension(64)]))
('x shape after bias_add ', TensorShape([Dimension(None), Dimension(23), Dimension(38), Dimension(64)]))


In [32]:
# Evaluate model
correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

## load data into queue

In [33]:
filename_queue = tf.train.string_input_producer(
    [tfrecords_train_filename])
test_queue = tf.train.string_input_producer(
    [tfrecords_test_filename])

In [34]:
# Even when reading in multiple threads, share the filename
# queue.
print "read_and_decode"
image, annotation, label = read_and_decode(filename_queue)
print(image.shape)

read_and_decode
image shape
(?,)
(1, 200, 320, 3)


In [35]:
# The op for initializing the variables.
print "init_op"
# init_op = tf.group(tf.global_variables_initializer(),
#                    tf.local_variables_initializer())
init = tf.global_variables_initializer()
print "done"

init_op
done


In [36]:
img.shape

(1, 200, 320, 3)

In [45]:
# config
# config = tf.ConfigProto()
# config.gpu_options.allow_growth = True
# Launch the graph
with tf.Session() as sess:
    print("started")
    sess.run(init)
    print("session run")
    coord = tf.train.Coordinator()
    print("coord")
    threads = tf.train.start_queue_runners(coord=coord)
    print("created threads")
    step=1
    print("training ...")
    # Keep training until reach max iterations
    while (step * batch_size) < training_iters:# and (not coord.should_stop):
        print("session run")
        img, anno, lab = sess.run([image, annotation, label])
        
        #img = tf.reshape(img, [1, n_input])
        # Run optimization op (backprop)
        #x = tf.placeholder(tf.float32, [1, 200, 320, 3])
        print(x.shape)
        sess.run(optimizer, feed_dict={x: img, y: lab,
                                       keep_prob: dropout})
        print("optimize")
        if step % display_step == 0:
            # Calculate batch loss and accuracy
#             img, anno, lab = sess.run([image, annotation, label])
            loss, acc = sess.run([cost, accuracy], feed_dict={x: img,
                                                              y: lab,
                                                              keep_prob: 1.})
            print("Iter " + str(step*batch_size) + ", Minibatch Loss= " + \
                  "{:.6f}".format(loss) + ", Training Accuracy= " + \
                  "{:.5f}".format(acc))
        step += 1
    print("Optimization Finished!")

    # Calculate accuracy for biome test images
    image, annotation, label =read_and_decode(test_queue)
    img, anno, lab = sess.run([image, annotation, label])
    print("Testing Accuracy:", \
        sess.run(accuracy, feed_dict={x: img,
                                      y: lab,
                                      keep_prob: 1.}))


started
session run
coord
created threads
training ...
session run
(1, 200, 320, 3)
INFO:tensorflow:Error reported to Coordinator: <class 'tensorflow.python.framework.errors_impl.CancelledError'>, Enqueue operation was cancelled
	 [[Node: input_producer_3/input_producer_3_EnqueueMany = QueueEnqueueManyV2[Tcomponents=[DT_STRING], timeout_ms=-1, _device="/job:localhost/replica:0/task:0/cpu:0"](input_producer_3, input_producer_3/RandomShuffle)]]

Caused by op u'input_producer_3/input_producer_3_EnqueueMany', defined at:
  File "/usr/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/usr/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python2.7/dist-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/usr/local/lib/pytho

ValueError: Cannot feed value of shape (1,) for Tensor u'Placeholder_13:0', which has shape '(?, 2)'

In [ ]:
img.shape

In [ ]:
lab.shape